#Student Name: Cormac Lavery
#Student ID: 16139658

## Etivity4 : Vectors, Matrics and Tensors

Building on my peers code found [here](https://gitlab.com/CE4021/Group2/week3/tree/e28267c1071e79b67d522f9f6d89659c6f650a14)

I have done the following:
 - converted functions to a matrix class
 - implemented addition/subtraction/scalar multiply and matrix multiply as overrides of the arithmetic operators: + - * @ 
 - I have provided additional functions to calculate:
    - the determinant (NxN matrices)
    - the inverse (only 2x2 matrices)
    - the cross product of two suitable vectors (only 3d vectors)
 - confirmed functionality with additional unit tests

On my peer Martin's review and additional advice from Mark and Brian I have made the following changes:
 - increased usability by changing class private methods to class and subclass private methods
 - improved error handling by raising exception for inverse matrices not of size 2x2
 - made the variable names more explicit (i.e. nc => number_columns)
 - allowed the calculation of cross products for tensor with rank < 3 by adding a pad to 3d vector
 - allowed for calculation of the determinant of 1 * 1 matrices


In [63]:
class Matrix:

    def __init__(self, rows) -> None:
        self.rows = rows
        self.shape = self._matrix_size()

    def _matrix_size(self):
        # Calculate the size of the tensor as a 2D tuple
        """Function that accepts a matrix in the form of a list of lists and returns
           a tuple indicating matrix size
           Returns (Number of Rows, Number of Columns)
        """
        number_rows = len(self.rows)
        number_columns = len(self.rows[0])
        return (number_rows, number_columns)

    def _matrix_init(self, size):
        """Function to initialize a matrix. Size received as a tuple of the form
           (Number of Rows, Number of Columns)
           Returns matrix in specified size as a list of lists
        """
        # Code taken from https://www.programiz.com/python-programming/matrix
        number_rows, number_columns = size
        n = [0] * number_rows
        for i in range(number_rows):
            n[i] = [0] * number_columns
        return n

    def __add__(self, other_matrix):
        return self._matrix_add_sub(other_matrix)

    def __sub__(self, other_matrix):
        return self._matrix_add_sub(other_matrix, op="sub")

    def _matrix_add_sub(self, m, op="add"):
        # Sum/subtract the tensor with another tensor of suitable size
        """Function to add or substract two matrices. Operation is specified using
        'op' string and defaults to addition
        """
        (number_rows, number_columns) = self.shape

        # Matrix Size Mismatch. Addition/Subtraction Not Possible
        if (m.shape) != (number_rows, number_columns):
            return "ERROR"
        else:
            result = self._matrix_init((number_rows, number_columns))

        for i in range(number_rows):
            for j in range(number_columns):
                if op == 'sub':
                    result[i][j] = self.rows[i][j] - m.rows[i][j]
                else:
                    result[i][j] = self.rows[i][j] + m.rows[i][j]
        rows = [tuple(i) for i in result]
        return Matrix(tuple(rows))

    def __mul__(self, m):
        # Multiply the tensor with a scalar
        """Function to multiply a matrix by a scalar.
        """

        number_rows, number_columns = self.shape
        result = self._matrix_init(self.shape)
        for i in range(number_rows):
            for j in range(number_columns):
                result[i][j] = self.rows[i][j] * m
        rows = (tuple(i) for i in result)
        return Matrix(tuple(rows))

    def __matmul__(self, m):
        # Multiply the tensor with a tensor of suitable size or tensor by a scalar
        """Function to multiply two matrices. If matrix dimensions prevent
        multiplication, ERROR is returned
        """

        (number_rows_n, nc_n) = self.shape
        (number_rows_m, nc_m) = m.shape

        # Number of Columns in Matrix N must equal Number of Rows in Matrix M
        if (nc_n != number_rows_m):
            raise ValueError("Number of Columns in Matrix N must equal Number of Rows in Matrix M")
        else:
            result = self._matrix_init((number_rows_n, nc_m))

        for i in range(number_rows_n):
            for j in range(nc_m):
                for k in range(nc_n):  # Could also have used number_rows_m as nc_n==number_rows_m
                    result[i][j] += self.rows[i][k] * m.rows[k][j]
        rows = (tuple(i) for i in result)
        return Matrix(tuple(rows))

    def __str__(self):
        return str(self.rows)

    def __eq__(self, other_matrix):
        return self.rows == other_matrix.rows

    def determinant(self):
        # Calculate the determinant of the tensor, but only for square tensors (matrix) I have used a recursive call
        # which can calculate the determinant of any square matrix by splitting the problem down
        (number_rows, number_columns) = self.shape
        if number_rows != number_columns:
            raise ValueError("implementation only covers square matrices")
        elif self.shape == (1, 1):
            return self.rows[0][0]
        elif self.shape == (2, 2):
            return (self.rows[0][0] * self.rows[1][1]) - (self.rows[1][0]) * self.rows[0][1]
        else:
            determinant = 0
            for i, column in enumerate(self.rows[0]):
                pivot = column
                pivot_matrix = self._get_matrix_excluding_row_and_column(0, i)
                determinant += ((-1) ** (2 + i)) * pivot * pivot_matrix.determinant()
            return determinant

    def _get_matrix_excluding_row_and_column(self, row_index, column_index):
        # Returns the minor matrix composed of all element not in specified row and column
        number_rows, number_columns = self.shape
        rows = []
        for i in range(number_rows):
            if row_index != i:
                row = []
                for j in range(number_columns):
                    if column_index != j:
                        row.append(self.rows[i][j])
                rows.append(tuple(row))
        return Matrix(tuple(rows))

    def inverse(self):
        # Calculate the inverse of the tensor, but only for tensors of size 2x2
        if self.shape != (2,2):
            raise ValueError("can only calculate the inverse of 2x2 matrices")

        determinant = self.determinant()
        if determinant == 0:
            return None
        a = self.rows[0][0] / determinant
        b = self.rows[0][1] / determinant
        c = self.rows[1][0] / determinant
        d = self.rows[1][1] / determinant
        return Matrix(((d, -b), (-c, a)))

    def cross_product(self, other_vector):
        # Calculate the cross product of the tensor with another tensor of suitable size
        # This implementation only works with vectors with rank 3

        (number_rows_n, nc_n) = self.shape
        (number_rows_m, nc_m) = other_vector.shape

        if number_rows_n > 3 or number_rows_m > 3 or nc_n != 1 or nc_m != 1:
            raise ValueError("can only calculate cross product for vectors of rank 3 or less")
        
        vector_a = self._pad_matrix_to_rank(3)
        vector_b = other_vector._pad_matrix_to_rank(3)
        
        rows = []
        for i in range(3):
            column = (0, vector_a.rows[i][0], vector_b.rows[i][0])
            rows.append(column)
        cross_product_matrix = Matrix(tuple(rows))

        iHat = Matrix(((1,), (0,), (0,)))
        jHat = Matrix(((0,), (1,), (0,)))
        kHat = Matrix(((0,), (0,), (1,)))

        a = iHat * cross_product_matrix._get_matrix_excluding_row_and_column(0, 0).determinant()
        b = jHat * -cross_product_matrix._get_matrix_excluding_row_and_column(1, 0).determinant()
        c = kHat * cross_product_matrix._get_matrix_excluding_row_and_column(2, 0).determinant()

        return a + b + c

    def _pad_matrix_to_rank(self, rank, value=0):
        # pads a matrix with additional elements of specified value until it has the required rank
        padded_rows = []
        for i in range(rank):
            if i < len(self.rows):
                padded_rows.append(tuple(self.rows[i]))
            else:
                padded_rows.append(tuple([value] * len(self.rows[0])))
        return Matrix(tuple(padded_rows))


In [62]:
##################################################
# Unit Test
##################################################
import unittest

a = Matrix(((-2, 7), (9, 4)))  # A: 2x2 Matrix
b = Matrix(((4, 8), (-6, 1))) # B: 2x2 Matrix
c = Matrix(((8, 2),))  # C: 1x2 Row Vector
d = Matrix(((-3,), (4,)))  # D: 2x1 Column Vector


class TestEtivity3(unittest.TestCase):
    # Test Matrix Addition and Subtraction
    def test_001(self):
        expected_result = Matrix(((2, 15), (3, 5)))  # A+B
        self.assertEqual(a + b, expected_result, "A+B Mismatch")

        expected_result = Matrix(((-6, -1), (15, 3)))  # A-B
        self.assertEqual(a - b, expected_result, "A-B Mismatch")


# Test Matrix Multiplication
    def test_002(self):
        expected_result = Matrix(((-50, -9), (12, 76)))  # A*B
        self.assertEqual(a @ b, expected_result, "A*B Mismatch")

        expected_result = Matrix(((64, 60), (21, -38))) # B*A
        self.assertEqual(b @ a, expected_result, "B*A Mismatch")

        expected_result = Matrix(((2, 64),))  # C*A
        self.assertEqual(c @ a, expected_result, "C*A Mismatch")

        expected_result = Matrix(((34,), (-11,)))  # A*D
        self.assertEqual(a @ d, expected_result, "A*D Mismatch")

        expected_result = Matrix(((-16,),))  # C*D
        self.assertEqual(c @ d, expected_result, "C*D Mismatch")

        expected_result = Matrix(((-24, -6), (32, 8)))  # D*C
        self.assertEqual(d @ c, expected_result, "D*C Mismatch")
        
        expected_result = Matrix(((-4, 14), (18, 8)))  # A*C
        self.assertEqual(a * 2, expected_result, "A*C Mismatch")

# Test Matrix determinant, inverse and cross product
    def test_003(self):
        matrix = Matrix(((5,7),(2,3)))
        expected_result = 1
        self.assertEqual(matrix.determinant(), expected_result)

        matrix = Matrix(((5,2),(-7,-3)))
        expected_result = Matrix(((3,2),(-7,-5)))
        self.assertEqual(matrix.inverse(), expected_result)

        matrix = Matrix(((2,-3,1),(2,0,-1),(1,4,5)))
        expected_result = 49
        self.assertEqual(matrix.determinant(), expected_result)

        vector_a = Matrix(((1,),(2,),(3,)))
        vector_b = Matrix(((3,),(2,),(1,)))
        expected_result = Matrix(((-4,),(8,),(-4,)))
        actual_result = vector_a.cross_product(vector_b)
        self.assertEqual(actual_result, expected_result)

        vector_a = Matrix(((1,),(2,),(3,)))
        expected_result = Matrix(((0,),(0,),(0,)))
        actual_result = vector_a.cross_product(vector_a)
        self.assertEqual(actual_result, expected_result)

        vector_a = Matrix(((2,),(3,),(4,)))
        vector_b = Matrix(((5,),(6,),(7,)))
        expected_result = Matrix(((-3,),(6,),(-3,)))
        actual_result = vector_a.cross_product(vector_b)
        self.assertEqual(actual_result, expected_result)

##Tests as required
    def test_required_tests_etivity_4(self):
        
        #sum two 4x4 matrices
        matrix_a = Matrix(((1,2,3,2),(3,2,1,3),(2,2,1,1),(1,2,1,1)))
        expected_matrix = Matrix(((2,4,6,4),(6,4,2,6),(4,4,2,2),(2,4,2,2)))
        self.assertEqual(matrix_a + matrix_a, expected_matrix)
        
        #multiply two 4x4 matrices
        matrix_a = Matrix(((1,2,3,2),(3,2,1,3),(2,2,1,1),(1,2,1,1)))
        matrix_b = Matrix(((1,0,0,1),(1,2,1,1),(0,0,1,1),(1,2,0,1)))
        expected_matrix = Matrix(((5,8,5,8), (8,10,3,9), (5,6,3,6), (4,6,3,5)))
        self.assertEqual(matrix_a @ matrix_b, expected_matrix)
        
        #multiply 4x4 matrix by suitable vector (rank 4 vector)
        matrix_a = Matrix(((1,2,3,2),(3,2,1,3),(2,2,1,1),(1,2,1,1)))
        vector_b = Matrix(((1,),(2,),(3,),(2,)))
        expected_matrix = Matrix(((18,),(16,),(11,),(10,)))
        self.assertEqual(matrix_a @ vector_b, expected_matrix)
        
        #determinant of a 2x2 matrix
        matrix_a = Matrix(((1,2),(3,2)))
        expected_result = -4
        self.assertEqual(matrix_a.determinant(), expected_result)
        
        #inverse of a 2x2 matrix
        matrix_a = Matrix(((1,2),(3,2)))
        expected_matrix = Matrix(((-0.5, 0.5), (0.75, -0.25)))
        self.assertEqual(matrix_a.inverse(), expected_matrix)

        #cross-product of 2 suitable tensors
        vector_a = Matrix(((2,),(3,),(4,)))
        vector_b = Matrix(((5,),(6,),(7,)))
        expected_result = Matrix(((-3,), (6,), (-3,)))
        actual_result = vector_a.cross_product(vector_b)
        self.assertEqual(actual_result, expected_result)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)


.

.

E

E


ERROR: test_003 (__main__.TestEtivity3)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-62-8d388c4e8ed5>", line 62, in test_003
    actual_result = vector_a.cross_product(vector_b)
  File "<ipython-input-61-8323700f31e4>", line 158, in cross_product
    column = (0, vector_a[i][0], vector_b.rows[i][0])
TypeError: 'Matrix' object does not support indexing

ERROR: test_required_tests_etivity_4 (__main__.TestEtivity3)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-62-8d388c4e8ed5>", line 110, in test_required_tests_etivity_4
    actual_result = vector_a.cross_product(vector_b)
  File "<ipython-input-61-8323700f31e4>", line 158, in cross_product
    column = (0, vector_a[i][0], vector_b.rows[i][0])
TypeError: 'Matrix' object does not support indexing

----------------------------------------------------------------------
Ran 4 t